In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
DATA_DIR = 'tmp/data'

In [3]:
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [34]:
def make_cnn(stride_size):
    W_conv1 = weight_variable([stride_size, stride_size, 1, 32])
    b_conv1 = bias_variable([32])
    
    x_image = tf.reshape(x, [-1,28,28,1])
    
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    # Second layer
    W_conv2 = weight_variable([stride_size, stride_size, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    # Fully connected layer
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    # keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    # y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [52]:
def train_and_test():
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        for i in range(n_steps):
            batch = data.train.next_batch(minibatch_size)
            if i % step_to_print == 0:
                train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
                print("step {}, training accuracy {}".format(i, train_accuracy))
            train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

        # eval test set 
        # Full doesn't fit in my RAM, so it's splitted into smaller batches
        test_acc = 0
        size = data.test.images.shape[0]
        n_batches = 4
        batch_size = size // n_batches
        for i in range(n_batches):
            test_acc += accuracy.eval(
                feed_dict={x: data.test.images[batch_size * i: batch_size * (i + 1)],
                           y_: data.test.labels[batch_size * i: batch_size * (i + 1)],
                           keep_prob: 1.0})
        print("test accuracy {}".format(test_acc / n_batches))

In [36]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [37]:
n_steps = 4000
minibatch_size = 100
step_to_print = 200

In [38]:
y_conv = make_cnn(5)

### Stride size is 5

In [31]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(n_steps):
        batch = data.train.next_batch(minibatch_size)
        if i % step_to_print == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    # eval test set 
    # Full doesn't fit in my RAM, so it's splitted into smaller batches
    test_acc = 0
    size = data.test.images.shape[0]
    n_batches = 4
    batch_size = size // n_batches
    for i in range(n_batches):
        test_acc += accuracy.eval(
            feed_dict={x: data.test.images[batch_size * i: batch_size * (i + 1)],
                       y_: data.test.labels[batch_size * i: batch_size * (i + 1)],
                       keep_prob: 1.0})
    
    print("test accuracy {}".format(test_acc / n_batches))

step 0, training accuracy 0.07999999821186066
step 200, training accuracy 0.9399999976158142
step 400, training accuracy 0.8999999761581421
step 600, training accuracy 0.9800000190734863
step 800, training accuracy 0.9200000166893005
step 1000, training accuracy 0.9399999976158142
step 1200, training accuracy 0.9399999976158142
step 1400, training accuracy 0.9800000190734863
step 1600, training accuracy 0.9800000190734863
step 1800, training accuracy 0.9599999785423279
step 2000, training accuracy 0.9800000190734863
step 2200, training accuracy 1.0
step 2400, training accuracy 0.9399999976158142
step 2600, training accuracy 0.9399999976158142
step 2800, training accuracy 1.0
step 3000, training accuracy 0.9599999785423279
step 3200, training accuracy 0.9800000190734863
step 3400, training accuracy 0.9599999785423279
step 3600, training accuracy 0.9800000190734863
step 3800, training accuracy 1.0
test accuracy 0.9843999892473221


### Stride size is 3

In [39]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [40]:
y_conv = make_cnn(3)

In [41]:
n_steps = 4000
minibatch_size = 100
step_to_print = 200

In [53]:
train_and_test()

step 0, training accuracy 0.03999999910593033
step 200, training accuracy 0.8999999761581421
step 400, training accuracy 0.9300000071525574
step 600, training accuracy 0.8999999761581421
step 800, training accuracy 0.9800000190734863
step 1000, training accuracy 0.9399999976158142
step 1200, training accuracy 0.9700000286102295
step 1400, training accuracy 0.9800000190734863
step 1600, training accuracy 0.9800000190734863
step 1800, training accuracy 0.9800000190734863
step 2000, training accuracy 0.9800000190734863
step 2200, training accuracy 0.9700000286102295
step 2400, training accuracy 0.9599999785423279
step 2600, training accuracy 0.9900000095367432
step 2800, training accuracy 0.9399999976158142
step 3000, training accuracy 0.9900000095367432
step 3200, training accuracy 1.0
step 3400, training accuracy 0.9700000286102295
step 3600, training accuracy 0.9900000095367432
step 3800, training accuracy 0.9900000095367432
test accuracy 0.9864999949932098
